In [11]:
import yfinance as yf
import pandas as pd
import numpy as np
from tqdm import tqdm

# Step 1: Define NIFTY 50 Tickers
nifty50_tickers = [
    "RELIANCE.NS", "TCS.NS", "INFY.NS", "HDFCBANK.NS", "ICICIBANK.NS",
    "HINDUNILVR.NS", "SBIN.NS", "ITC.NS", "LT.NS", "KOTAKBANK.NS",
    "AXISBANK.NS", "BHARTIARTL.NS", "ASIANPAINT.NS", "BAJFINANCE.NS",
    "HCLTECH.NS", "MARUTI.NS", "SUNPHARMA.NS", "NTPC.NS", "WIPRO.NS",
    "ULTRACEMCO.NS", "POWERGRID.NS", "NESTLEIND.NS", "TITAN.NS",
    "JSWSTEEL.NS", "TATAMOTORS.NS", "COALINDIA.NS", "TATASTEEL.NS",
    "GRASIM.NS", "ADANIENT.NS", "TECHM.NS", "INDUSINDBK.NS", "BPCL.NS",
    "CIPLA.NS", "DRREDDY.NS", "BRITANNIA.NS", "ONGC.NS", "HDFCLIFE.NS",
    "BAJAJFINSV.NS", "DIVISLAB.NS", "BAJAJ-AUTO.NS", "HINDALCO.NS",
    "SBILIFE.NS", "HEROMOTOCO.NS", "EICHERMOT.NS", "SHREECEM.NS",
    "UPL.NS", "TATACONSUM.NS", "M&M.NS", "APOLLOHOSP.NS", "LTIM.NS"
]

# Step 2: Initialize empty DataFrame
columns = [
    "Ticker", "Market Cap", "Beta", "FCF", "Growth Rate", "Debt", "Equity",
    "Cost of Debt", "Tax Rate", "Cost of Equity", "WACC", "Intrinsic Value"
]
financial_data = pd.DataFrame(columns=columns)

# Step 3: Constants
risk_free_rate = 0.07
market_return = 0.12
cost_of_debt_assumption = 0.08
tax_rate = 0.25
growth_rate = 0.05  # conservative long-term growth

# Step 4: Loop through tickers
for ticker in tqdm(nifty50_tickers, desc="Processing NIFTY50 companies"):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        market_cap = info.get("marketCap", None)
        beta = info.get("beta", None)
        fcf = info.get("freeCashflow", None)
        total_debt = info.get("totalDebt", 0)

        if not all([market_cap, beta, fcf]):
            print(f"⚠️ Skipping {ticker} due to missing financial data.")
            continue

        market_cap = float(market_cap)
        beta = float(beta)
        fcf = float(fcf)
        total_debt = float(total_debt)
        equity = market_cap
        cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)
        cost_of_debt = cost_of_debt_assumption

        total_value = equity + total_debt
        wacc = ((equity / total_value) * cost_of_equity +
                (total_debt / total_value) * cost_of_debt * (1 - tax_rate))

        # DCF using Gordon Growth Model
        if wacc <= growth_rate or fcf <= 0:
            intrinsic_value = np.nan
        else:
            intrinsic_value = (fcf * (1 + growth_rate)) / (wacc - growth_rate)

        financial_data = pd.concat([financial_data, pd.DataFrame([{
            "Ticker": ticker,
            "Market Cap": market_cap,
            "Beta": beta,
            "FCF": fcf,
            "Growth Rate": growth_rate,
            "Debt": total_debt,
            "Equity": equity,
            "Cost of Debt": cost_of_debt,
            "Tax Rate": tax_rate,
            "Cost of Equity": cost_of_equity,
            "WACC": wacc,
            "Intrinsic Value": intrinsic_value
        }])], ignore_index=True)

    except Exception as e:
        print(f"❌ Error processing {ticker}: {e}")

# Step 5: Filter out invalid values
financial_data.dropna(subset=["Intrinsic Value"], inplace=True)

# Step 6: Weighting by market cap
financial_data["Weight"] = financial_data["Market Cap"] / financial_data["Market Cap"].sum()
financial_data["Weighted Intrinsic Value"] = financial_data["Intrinsic Value"] * financial_data["Weight"]
nifty50_intrinsic_value = financial_data["Weighted Intrinsic Value"].sum()

# Step 7: Display results (clean formatting)
pd.set_option('display.float_format', lambda x: f'₹{x:,.2f}')
print("\n===== NIFTY 50 DCF Analysis Result (Corrected) =====\n")
print(financial_data[columns + ["Weight"]].sort_values(by="Intrinsic Value", ascending=False))

print(f"\n🧮 Estimated Intrinsic Value of NIFTY 50 Index: ₹{nifty50_intrinsic_value:,.2f}")

Processing NIFTY50 companies:   2%|▎             | 1/50 [00:00<00:11,  4.40it/s]

⚠️ Skipping RELIANCE.NS due to missing financial data.


/var/folders/6v/9qg480fs1tl47ks4b9pr9dmr0000gn/T/ipykernel_53522/3146843494.py:68: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  financial_data = pd.concat([financial_data, pd.DataFrame([{
Processing NIFTY50 companies:   8%|█             | 4/50 [00:00<00:10,  4.29it/s]

⚠️ Skipping HDFCBANK.NS due to missing financial data.


Processing NIFTY50 companies:  10%|█▍            | 5/50 [00:01<00:10,  4.41it/s]

...

⚠️ Skipping M&M.NS due to missing financial data.


Processing NIFTY50 companies:  98%|████████████▋| 49/50 [00:10<00:00,  4.63it/s]

⚠️ Skipping APOLLOHOSP.NS due to missing financial data.


Processing NIFTY50 companies: 100%|█████████████| 50/50 [00:10<00:00,  4.56it/s]


===== NIFTY 50 DCF Analysis Result (Corrected) =====

          Ticker             Market Cap  Beta                 FCF  \
0         TCS.NS ₹11,937,526,054,912.00 ₹0.45 ₹346,417,496,064.00   
4       WIPRO.NS  ₹2,576,371,941,376.00 ₹0.66 ₹126,595,497,984.00   
2  BHARTIARTL.NS ₹10,447,031,894,016.00 ₹0.19  ₹58,636,124,160.00   
6        LTIM.NS  ₹1,225,302,081,536.00 ₹0.59  ₹37,695,250,432.00   
5     DRREDDY.NS    ₹924,757,000,192.00 ₹0.48  ₹11,819,374,592.00   
1        INFY.NS  ₹6,014,302,158,848.00 ₹0.57   ₹3,050,374,912.00   
3     HCLTECH.NS  ₹3,854,274,920,448.00 ₹0.49   ₹2,214,749,952.00   

   Growth Rate                  Debt                 Equity  Cost of Debt  \
0        ₹0.05    ₹93,510,000,640.00 ₹11,937,526,054,912.00         ₹0.08   
4        ₹0.05   ₹193,559,003,136.00  ₹2,576,371,941,376.00         ₹0.08   
2        ₹0.05 ₹2,046,230,003,712.00 ₹10,447,031,894,016.00         ₹0.08   
6        ₹0.05    ₹22,357,000,192.00  ₹1,225,302,081,536.00         ₹0.08   
5      

In [13]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm

# Get NIFTY50 tickers
nifty50_tickers = [
    'RELIANCE.NS', 'HDFCBANK.NS', 'ICICIBANK.NS', 'INFY.NS', 'HINDUNILVR.NS',
    'SBIN.NS', 'ITC.NS', 'LT.NS', 'KOTAKBANK.NS', 'AXISBANK.NS',
    'ASIANPAINT.NS', 'BAJFINANCE.NS', 'TCS.NS', 'MARUTI.NS', 'SUNPHARMA.NS',
    'NTPC.NS', 'ULTRACEMCO.NS', 'POWERGRID.NS', 'NESTLEIND.NS', 'TITAN.NS',
    'JSWSTEEL.NS', 'TATAMOTORS.NS', 'COALINDIA.NS', 'TATASTEEL.NS',
    'GRASIM.NS', 'ADANIENT.NS', 'TECHM.NS', 'INDUSINDBK.NS', 'HCLTECH.NS',
    'BPCL.NS', 'CIPLA.NS', 'WIPRO.NS', 'BAJAJ-AUTO.NS', 'DRREDDY.NS',
    'BHARTIARTL.NS', 'BRITANNIA.NS', 'ONGC.NS', 'HDFCLIFE.NS',
    'BAJAJFINSV.NS', 'DIVISLAB.NS', 'EICHERMOT.NS', 'HINDALCO.NS',
    'SBILIFE.NS', 'ADANIPORTS.NS', 'APOLLOHOSP.NS', 'UPL.NS', 'HEROMOTOCO.NS',
    'TATACONSUM.NS', 'BAJAJHLDNG.NS', 'SHREECEM.NS'
]

# Initialize dataframe to store all financial data
financial_data = []

# Loop through each ticker and fetch info safely
print("Fetching financial data for NIFTY50 companies...\n")
for ticker in tqdm(nifty50_tickers, desc="Processing NIFTY50 companies"):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        # Skip if financial info is missing or empty
        if not info or info.get("shortName") is None:
            print(f"⚠️ Skipping {ticker} due to missing financial data.")
            continue

        # Extract key financial metrics
        data = {
            'Ticker': ticker,
            'Name': info.get('shortName'),
            'Sector': info.get('sector'),
            'Industry': info.get('industry'),
            'Market Cap': info.get('marketCap'),
            'PE Ratio': info.get('trailingPE'),
            'EPS': info.get('trailingEps'),
            'Price to Book': info.get('priceToBook'),
            'Dividend Yield': info.get('dividendYield'),
            'Return on Equity': info.get('returnOnEquity'),
            'Current Price': info.get('currentPrice')
        }

        financial_data.append(data)

    except Exception as e:
        print(f"⚠️ Skipping {ticker} due to error: {e}")

# Convert to DataFrame only if data is available
if financial_data:
    df_financials = pd.DataFrame(financial_data)
    print("\n✅ Financial data collected successfully!")
    display(df_financials.head())
else:
    print("❌ No financial data collected. Please check tickers or try again later.")

Fetching financial data for NIFTY50 companies...



Processing NIFTY50 companies: 100%|█████████████| 50/50 [00:12<00:00,  4.05it/s]


✅ Financial data collected successfully!


,Ticker,Name,Sector,Industry,Market Cap,PE Ratio,EPS,Price to Book,Dividend Yield,Return on Equity,Current Price
0,RELIANCE.NS,RELIANCE INDUSTRIES LTD,Energy,Oil & Gas Refining & Marketing,16302481604608,₹23.58,₹51.08,₹1.99,₹0.42,NaN,"₹1,204.70"
1,HDFCBANK.NS,HDFC BANK LTD,Financial Services,Banks - Regional,13906342838272,₹20.33,₹89.41,₹2.76,₹1.07,₹0.15,"₹1,817.30"
2,ICICIBANK.NS,ICICI BANK LTD.,Financial Services,Banks - Regional,9509914279936,₹19.46,₹68.63,₹3.22,₹0.75,₹0.19,"₹1,335.30"
3,INFY.NS,INFOSYS LIMITED,Technology,Information Technology Services,6014302158848,₹21.46,₹67.65,₹583.46,₹2.82,₹0.33,"₹1,451.65"
4,HINDUNILVR.NS,HINDUSTAN UNILEVER LTD.,Consumer Defensive,Household & Personal Products,5273772621824,₹49.02,₹45.79,₹10.38,₹1.92,NaN,"₹2,244.55"


In [15]:
df_financials.to_csv("nifty50_financials.csv", index=False)